<a href="https://colab.research.google.com/github/JoseEliasDuranRoa/Prediction-Models-for-Housing-Prices-in-Chennai/blob/main/ProblemSet1_Dur%C3%A1nRoa_Jos%C3%A9El%C3%ADas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Please make a copy of this notebook and replace "LastName" and "FirstName" in the title with your own names. Also, write your name in the following text block.

* Then, fill in the file with your code and comments on the results.

* Finally, share your completed work with me by sending the link via email as your final submission, no later than **January 6th, 2025**.

* You can contact me if you have any questions. If necessary, we can meet either face-to-face or via online video calls to resolve them.

---

**Last Name:** Durán Roa


**First Name:** José Elías



---

# Problem 1: Prediction




In the shared subfolder "Problem1" you will find .zip files with datasets (and information about them). You must fill the "Choose_Dataset" file with your choice. You *cannot* choose an option previouly selected by somebody else. But note that in some cases you can choose among different subsamples of the same dataset.

## a) Data Cleaning

* Read the data; pay attention to use the right data type for each variable.

In [6]:
from google.colab import drive

In [7]:
#Import modules
import zipfile, os
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
pd.__version__

# Import the data
zip_file_path = '/content/drive/My Drive/chennai.zip'
housing = pd.read_csv(zipfile.ZipFile(zip_file_path).extract('Chennai_housing_sale_2.csv'), index_col = 0)

#Correct data type
housing['DATE_SALE'] = pd.to_datetime(housing['DATE_SALE'], dayfirst=True)
housing['DATE_BUILD'] = pd.to_datetime(housing['DATE_BUILD'], dayfirst=True)
housing['N_BEDROOM'] = housing['N_BEDROOM'].astype('Int64')
housing['N_BATHROOM'] = housing['N_BATHROOM'].astype('Int64');

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/chennai.zip'

* At this point, we have successfully loaded data related to housing in Chennai, a major city in India, into memory. This data is organized in a DataFrame with 21 distinct variables, which include information about property characteristics, sale prices, construction dates, and various amenities associated with the houses.

* The main purpose of this work is to predict housing prices using various predictors related to property characteristics and market conditions. As an initial excercise, to visualize the distribution of housing prices, we will create a histogram that highlights the frequency of different price ranges.

In [ ]:
# House prices distribution

fig = px.histogram(
    housing,
    x='SALES_PRICE',
    nbins=100,
    title='<b>Distribution of Housing Prices',
    labels={'SALES_PRICE': 'Housing Price'},
)

fig.update_layout(
    template='plotly_dark',
    title='<b> Figure 1: House Price Distribution in Chennai',
    xaxis_title='House price (Millions)',
    yaxis_title='Frequency',
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    bargap=0.2
)

fig.show()

#### Figure 1: Analysis


The histogram depicts the distribution of house prices in Chennai. The distribution is right-skewed, meaning higher prices are less frequent, with a peak around 10 million. An interesting observation is that the market has a clear concentration in mid-range prices, while luxury properties, priced above 20 million, are significantly fewer. This pattern is quite common in real estate markets globally, where most homes fall within the affordable to mid-range bracket, and high-end properties are relatively scarce. However, compared to some cities with broader income disparities, the price range here might suggest a more evenly distributed housing market, at least within this segment.  

In [ ]:
# Generate a label dictionary for posterior data visualization

labels = {
    'PRT_ID': 'House ID',
    'AREA': 'Location',
    'INT_SQFT': 'Area (sqft)',
    'DATE_SALE': 'Sale Date',
    'DIST_MAINROAD': 'Distance to Main Road',
    'N_BEDROOM': 'Number of Bedrooms',
    'N_BATHROOM': 'Number of Bathrooms',
    'N_ROOM': 'Total Rooms',
    'SALE_COND': 'Sale Condition',
    'PARK_FACIL': 'Parking Facility',
    'DATE_BUILD': 'Construction Date',
    'BUILDTYPE': 'Building Type',
    'UTILITY_AVAIL': 'Utility Availability',
    'STREET': 'Street Condition',
    'MZZONE': 'Market Zone',
    'QS_ROOMS': 'Room Quality',
    'QS_BATHROOM': 'Bathroom Quality',
    'QS_BEDROOM': 'Bedroom Quality',
    'QS_OVERALL': 'Overall Quality',
    'REG_FEE': 'Registration Fee',
    'COMMIS': 'Commission Fee',
    'SALES_PRICE': 'Sale Price'
};

In [ ]:
# Stastical Description of Data

describe_df = housing.describe()
describe_df = describe_df.drop(['DATE_SALE', 'DATE_BUILD'], axis=1)
describe_df.rename(columns=labels, inplace=True)

fig = px.imshow(describe_df,
                 text_auto=True,
                 aspect="auto",
                 color_continuous_scale='blues')

fig.update_layout(
    title="Table 1: Descriptive Statistics of Housing Data in Chennai",
    paper_bgcolor='black',
    plot_bgcolor='black',
    font_color='white',
    width=1000,
    height=400
)

fig.show()

#### Table 1: Analysis

The summary statistics suggest that, in Chennai’s housing market, property prices vary significantly more than size or quality, implying that factors like location or demand are likely driving price disparities. Most homes are close to main roads, highlighting the importance of accessibility in price determination. Interestingly, while house sizes and quality scores are relatively consistent, price variation is substantial, indicating that higher prices don’t necessarily reflect better quality. Transaction costs, such as registration and commission fees, also scale with price, adding a proportional burden to higher-priced homes, which could further widen affordability gaps.




* Now, we proceed to identify and correct typos in the categorical columns of the housing dataset. Next, we check for any missing values and print the columns that contain them.



In [5]:
# Identyfing AREA typos
housing.AREA.value_counts()
housing.BUILDTYPE.value_counts()
housing.UTILITY_AVAIL.value_counts()
housing.SALE_COND.value_counts()
housing.PARK_FACIL.value_counts()
housing.STREET.value_counts()

#Fyxing Typos
housing.AREA = housing.AREA.replace(
    {
        'Ana Nagar': 'Anna Nagar',
        'Ann Nagar': 'Anna Nagar',
        'Karapakkam': 'Karapakam',
        'Chrompt': 'Chrompet',
        'Chrmpet': 'Chrompet',
        'Chormpet': 'Chrompet',
        'KKNagar': 'KK Nagar',
        'TNagar': 'T Nagar',
        'Adyr': 'Adyar',
        'Velchery': 'Velachery'
    }
)
housing.BUILDTYPE = housing.BUILDTYPE.replace(
    {'Comercial': 'Commercial', 'Other': 'Others'}
)
housing.UTILITY_AVAIL = housing.UTILITY_AVAIL.replace(
    {'AllPub': 'All Pub', 'NoSewr': 'NoSeWa', 'NoSewr ': 'NoSeWa'}
)
housing.SALE_COND = housing.SALE_COND.replace(
    {'Ab Normal': 'AbNormal', 'PartiaLl': 'Partial', 'Partiall': 'Partial', 'Adj Land': 'AdjLand'}
)
housing.PARK_FACIL = housing.PARK_FACIL.replace('Noo', 'No')
housing.STREET = housing.STREET.replace(
    {'Pavd': 'Paved', 'NoAccess': 'No Access'}
)

#Checking missing values
missing_values = housing.isnull().sum()
print(missing_values[missing_values > 0])

NameError: name 'housing' is not defined

## b)

* Create a Pandas series your the outcome variable and a Pandas data frame with those variables that you consider as relevant predictors (probably, most of them). Additionally, you can think about transforming some of them (eg., join categories with few observations, generating dummies, etc.).

* Split your sample into a training set and a testing set.


In [ ]:
# Your code here

## c)

* Estimate a linear regression model by OLS using your preferred specification or specifications for prediction. You can use pre-processing steps if necessary (for polynomials, interactions, dealing with NAs, etc.). Comment on the results.

* Obtain the RMSE and the MAE as metrics of predictive performance of the model(s).



In [ ]:
# Your code here


> Your comments here


## d)

* Estimate a LASSO model using the pre-processing steps that you consider necessary. Note that you must choose the optimal value of the penalization hyperparameter by cross-validation before estimating the best LASSO model (which use that value)

* Comment on the results.

* Obtain the RMSE and the MAE as metrics of predictive performance of the model.

In [ ]:
# Your code here


> Your comments here


## e)

* Estimate a regression tree model using the pre-processing steps that you consider necessary. Note that you must choose the optimal value of the hyperparameters by cross-validation before estimating the best tree model. You can focus on tuning the cost-complexity parameter; you can fix the `min_samples_leaf` to a value of your choice (you can try a few of them before choosing one).

* Show a figure with the resulting tree. Comment on the results.

* Show in a table the variable importance according to this model (you can also provide a figure if you want). Comment on the results.

* Obtain the RMSE and the MAE as metrics of predictive performance of the model.

In [ ]:
# Your code here


> Your comments here


## f)

* Estimate a random forest model using the pre-processing steps that you consider necessary. Note that you must choose the optimal value of the hyperparameters by cross-validation before estimating the best model. You can focus on tuning either `min_samples_leaf` or `max_features`, but you should try a few of the other one.

* Show in a table the variable importance according to this model (you can also provide a figure if you want). Comment on the results.

* Obtain the RMSE and the MAE as metrics of predictive performance of the model.

In [ ]:
# Your code here


> Your comments here


## g)

Finally, discuss which prediction model is the best one. Also, briefly comment on any other key insights you can learn from one or more of these models.



> Your comments here


---

# Problem 2: Financial Investment

* This problem is **optional**. Completing it will give you 1 extra point.

* We are interested in estimating the effect of participation in 401(k) pension plans on employees’ net financial assets. The dataset [pension.csv](https://raw.githubusercontent.com/albarran/00datos/refs/heads/main/pension.csv) contains information about net financial assets (*net_tfa*) and participation in a 401(k) plan (*p401*), along with several additional individual characteristics: age, income, education, family size, marital status, having two earners in the household, having a defined benefit (DB) pension (a type of retirement plan where an employer guarantees a specified retirement amount), participation in an IRA (Individual Retirement Account, a type of retirement savings account with tax advantages), and homeownership. You can find more information about the variables [here](https://search.r-project.org/CRAN/refmans/hdm/html/pension.html).

* Since saving preferences (which are unobservable) can directly affect net asset holdings and the decision to participate, and may also be correlated with observable characteristics (e.g., income), participation in 401(k) plans can be endogenous. However, it is possible to exploit randomness (conditional on confounding variables) in eligibility for 401(k) plans (*e401*). An instrumental variable (IV) approach could be used to estimate the causal effect, but in this case, we will focus on estimating the intention-to-treat effect.


* Load the data and check the variables types.

In [ ]:
# Your code here

## a)

* Estimate using OLS a model to examine the effect of 401(k) eligibility on asset holdings, controlling for relevant confounding factors.

* Carefully consider how each variable should enter the model (e.g., whether education should be treated as a continuous or categorical variable). Additionally, consider whether polynomial terms or interaction effects might be useful. There is no universally correct choice, but your decisions should be justified and applied consistently throughout the analysis.

* Show the estimated intention-to-treat effect and its standard error and other relevant information to comment on the evidence on the results.

In [ ]:
# Your code here


> Your comments here


## b)

* Repeat the part a) but estimating now the model using LASSO.

In [ ]:
# Your code here


> Your comments here


## c)

* Repeat the part a) again but using Double-selection LASSO.

* Comment on the differences with the previous results and discuss your overall conclusion about the evidence.

In [ ]:
# Your code here


> Your comments here


## d)

* Finally repeat the estimation of the ITT using Double ML with Random Forest.

* Show the results and discuss

In [ ]:
# Your code here


> Your comments here